In [42]:
import soundfile as sf
import librosa
from p_tqdm import p_map

In [43]:
def get_rms(file):
    try:
        audio = sf.read(file)[0]
        rms = librosa.feature.rms(y=audio)*1000
        return {
            "file": file,
            "rms": rms.mean()
        }
    except:
        print(file)
        return {
            "file": file,
            "rms": None
        }

In [44]:
# import json

# train_files = [json.loads(line)["audio_filepath"] for line in open("/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/metadata_train/total_data_cls03_cmc_train.json").read().split("\n") if line]
# len(train_files)
import pandas as pd
train_file = "/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/notebooks/results_csv/final_train_addvad.csv"
train_df = pd.read_csv(train_file)
train_files = train_df["file"]
train_df.shape

(2467507, 3)

In [45]:
results = p_map(get_rms, train_files, num_cpus=20)

  0%|          | 0/2467507 [00:00<?, ?it/s]

In [46]:
filtered = [r["file"] for r in results if r["rms"] is not None and r["rms"] < 1]
print(len(filtered))

396


In [47]:
df_valid_filtered = train_df[~train_df['file'].isin(filtered)]
print(train_df.shape[0] - df_valid_filtered.shape[0])
print(df_valid_filtered.shape[0])
df_valid_filtered.to_csv("/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/notebooks/results_csv/total_data_rms_filter_train.csv", index=False)

396
2467111


In [11]:
import pandas as pd
pd.DataFrame(filtered).to_csv("rms_filter.csv", index=False)

In [9]:
from IPython.display import Audio
import IPython.display as ipd
ipd.display(Audio(filtered[1]['file'], autoplay=True))

In [17]:
csv_filter = "/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/notebooks/rms_filter.csv"
list_remove = pd.read_csv(csv_filter)["file"]

train_cls03 = "/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/notebooks/results_csv/add_data_cls03_1_train.csv"
train_cmc = "/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/notebooks/results_csv/data_train_filter_error.csv"
valid_cls03 = "/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/notebooks/results_csv/add_data_cls03_1_valid.csv" 
valid_cmc = "/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/notebooks/results_csv/data_valid_filter_error.csv"

df_train_cls03 = pd.read_csv(train_cls03)
df_valid_cls03 = pd.read_csv(valid_cls03)
df_train_cmc = pd.read_csv(train_cmc)
df_valid_cmc = pd.read_csv(valid_cmc)

train_csv = pd.concat([df_train_cls03,df_train_cmc])
valid_csv = pd.concat([df_valid_cls03,df_valid_cmc])

print(len(train_csv),len(valid_csv))

2467509 88472


In [18]:
df_train_filtered = train_csv[~train_csv['file'].isin(list_remove)] #lọc các file nằm trong list_remove
df_valid_filtered = valid_csv[~valid_csv['file'].isin(list_remove)] #lọc các file nằm trong list_remove

print(len(df_train_filtered),len(df_valid_filtered))

2466663 88472


In [19]:
2467509-2466663

846

In [20]:
df_train_filtered.to_csv("/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/notebooks/results_csv/total_data_rms_filter_train.csv", index=False)
df_valid_filtered.to_csv("/home/lmnguyen/Projects/cmc_stt/finetune-fast-conformer/notebooks/results_csv/total_data_rms_filter_valid.csv", index=False)

In [2]:
import wave 
import os

def get_duration(file):
    with wave.open(file, 'r') as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
    return duration

from pathlib import Path

wav_files = list(Path('/home/ntdong/Data/STT_dataset/10.telesale/audio_telesale').rglob('*.wav'))  
len(wav_files)
durations = [get_duration(str(f)) for f in wav_files if os.path.isfile(str(f))]

In [4]:
len(durations), sum(durations) / 3600

(80818, 61.20279420138787)